In [ ]:
import tweepy
import pandas as pd
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGfrlgEAAAAAO%2FyiySOwdbuECDZAcJOUua%2FocbA%3DfJbRxABJwM8hloqjQtjMb3UN96nbK3nTNrk5eoJcpAWZhJPvO9')
query = '@lakme -is:retweet lang:en'
tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=10000)

def get_label(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity == 0:
        return 'neutral'
    else:
        return 'negative'

data = []
for tweet in tweets:
    row = {
        'text': tweet.text, 
        'created_at': tweet.created_at,
        'polarity': get_sentiment(tweet.text),
        'label': get_label(get_sentiment(tweet.text))
    }
    data.append(row)
df = pd.DataFrame(data)
df.to_csv('/Users/aryanthakur/Downloads/Paytmcare.csv', index=False)

In [8]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import numpy as np

In [21]:
nlp = spacy.load("en_core_web_sm")

In [11]:
df=pd.read_csv('/Users/aryanthakur/Downloads/Paytmcare.csv')
df.head()

,text,created_at,polarity,label
0,"@Paytm , @Paytmcare where is your lite fast ...",2023-03-29 06:46:15+00:00,0.200,positive
1,I am extremely disappointed and frustrated wit...,2023-03-29 06:44:28+00:00,-0.725,negative
2,"I was paying 1000 rupees on 20th March,2023 to...",2023-03-29 06:44:08+00:00,-0.250,negative
3,@PaytmHelpcare0 Who is this replying your fake...,2023-03-29 06:41:35+00:00,-0.500,negative
4,I made a recharge on DMRC website for metro ca...,2023-03-29 06:40:48+00:00,0.000,neutral


In [14]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)
df['url_text'] = df['url_text'].apply(remove_mentions)
df['url_text'].head()

0     ,   where is your lite fast  payment is still...
1    I am extremely disappointed and frustrated wit...
2    I was paying 1000 rupees on 20th March,2023 to...
3              Who is this replying your fake account 
4    I made a recharge on DMRC website for metro ca...
Name: url_text, dtype: object

In [15]:
df['lc_text'] = df['url_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['lc_text'].head()

0    , where is your lite fast payment is still on ...
1    i am extremely disappointed and frustrated wit...
2    i was paying 1000 rupees on 20th march,2023 to...
3               who is this replying your fake account
4    i made a recharge on dmrc website for metro ca...
Name: lc_text, dtype: object

In [16]:
df['lcrp_text'] = df['lc_text'].str.replace('[^\w\s]','')
df['lcrp_text'].head()

/var/folders/6w/dtmngg7156vdm9brvnbc_nq40000gn/T/ipykernel_2193/1341007855.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lcrp_text'] = df['lc_text'].str.replace('[^\w\s]','')


0     where is your lite fast payment is still on p...
1    i am extremely disappointed and frustrated wit...
2    i was paying 1000 rupees on 20th march2023 to ...
3               who is this replying your fake account
4    i made a recharge on dmrc website for metro ca...
Name: lcrp_text, dtype: object

In [17]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['lcrpe_text'] = df['lcrp_text'].apply(lambda x: remove_emoji(x))

In [19]:
stop = stopwords.words('english')
df['lcrpestp_text'] = df['lcrpe_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['lcrpestp_text'].head()

0                  lite fast payment still pending syq
1    extremely disappointed frustrated closing acco...
2    paying 1000 rupees 20th march2023 paytm regist...
3                                replying fake account
4    made recharge dmrc website metro card using pa...
Name: lcrpestp_text, dtype: object

In [23]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
df['lcrpestpl_text']= df['lcrpestp_text'].apply(space)
df['lcrpestpl_text'].head()

0                    lite fast payment still pende syq
1    extremely disappointed frustrated closing acco...
2    pay 1000 rupee 20th march2023 paytm register m...
3                                   reply fake account
4    make recharge dmrc website metro card use payt...
Name: lcrpestpl_text, dtype: object

In [24]:
df.to_csv('/Users/aryanthakur/Downloads/PaytmcarePrePro.csv', index=False)